# Probability and Simulation

Now it may turn out that you cannot easily describe your problem analytically or enumerate all the possibilities in which case you can turn to simulation to do you event generation and counting, to get the probabilities you're interested in. Here's an example that actually happened to me.

## Cities

Some data scientists are involved in personalization at online retailers. They apply personalization models to different views of the products whether they are sales emails or on the website; anywhere there are default lists of products. Of course, not being content to rest on their laurels, these data scientists are constantly developing new personalization models. In order to test whether a new model is better than an old model, they engage in A/B testing. We'll talk about A/B testing later in the book but for now, A/B testing just means dividing your, ahem, test subjects into two groups *completely at random*. Group A will get the *control* or existing personalization model. Group B will get the *treatment* or new personalization model. That sets up the context for the following problem in computational probability.

Suppose you are one of those data scientists and have a personalization model that is being tested in 20 cities nationwide. Each city has 100,000 customers on the mailing list and you send an email every day.

The test has been running a few days and it looks like the new model (called the "treatment") is about 10% better than the current practice ("control"). Let us assume this is true (we'll see later how to test this). After another week of testing, the director of marketing comes to you and is concerned that the new model has done very poorly in one of the cities 3 days in a row and opines that it must not really be 10% better.

Is the director of marketing right? Does this mean the new model is worse than the current one? Maybe, maybe not. Deciding which model is better is a problem in inference. This isn't really what the director of marketing is asserting, though. They're asserting that the new model can't be better if there is a losing streak of 3 days in one city. Now this *is* a question we can answer now, using computational probability.

What we want to know--what the stakeholder needs to know--is, if the new model is definitely 10% better, what is the probability of seeing a string of 3 *worse* outcomes in a particular city? If the probability is high, then we needn't worry. If the probability is low, we might be concerned about test.

## What do we know?

* 20 cities
* Each city has 100,000 subscribers with the lists split in half for control and treatment.
* the purchase rate for control is 0.0001 (0.01%)
* the purchase rate for treatment is 0.00011 (0.011%)
* the lift is 0.011/0.010 = 1.1 - 1.0 = 10%

These probably seem small but daily purchase rates are often small.

Let's start out by simulating a single day's worth of purchases in a single city. We know what the ideal purchase rate is, but it's not going to pan out to be the same exact thing everyday. We need to simulate those purchases and calculate the actual purchase rate:

In [1]:
from random import random, seed

In [2]:
seed(128934662)

In [3]:
def actual_purchase_rate( population, purchase_rate):
    purchases = [1.0 if random() < purchase_rate else 0.0 
                 for i in range( population)]
    return sum( purchases)/population

Let's see how it does:

In [4]:
for _ in range( 5):
    print( actual_purchase_rate( 50000, 0.0001))

0.00012
8e-05
6e-05
0.00016
0.00012


These look reasonable.

Now let's simulate a comparison in outcomes for control and treatment in a city. If the control is better, we'll say that's "1.0" and if the treatment is better, we'll say that's "0.0". Additionally, half of each mailing list gets control and half of each mailing list gets the treatment:

In [5]:
def difference_in_purchase_rates(population, control_rate, treatment_rate):
    control_actual = actual_purchase_rate( population//2, control_rate)
    treatment_actual = actual_purchase_rate( population//2, treatment_rate)
    difference = 1.0 if control_actual > treatment_actual else 0.0
    return difference

Now let's see how that looks:

In [6]:
for _ in range( 5):
    print( difference_in_purchase_rates( 100000, 0.0001, 0.00011))

0.0
0.0
1.0
0.0
0.0


Now we want to see what happens over N days:

In [7]:
def simulate_difference_for_n_days(population, control_rate, treatment_rate, days):
    return [difference_in_purchase_rates( population, control_rate, treatment_rate)
            for i in range( days)]

Let's see what it looks like over 30 days:

In [8]:
print( simulate_difference_for_n_days( 100000, 0.0001, 0.00011, 30))

[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0]


There are a number of ways we might interpret the idea of a "streak".

* A streak is N 1's followed by a 0. For example, if N is 3, then we're looking for something like 0, 1, 1, 1, 0.
* A streak is N or more 1's followed by a 0. For example, if N is 3, then we're looking for 0, 1, 1, 1, 0 but also 0, 1, 1, 1, 1, 0.
* A streak is any number of N 1's for example if N is 3 and there are 5 1's, then that's 3 streaks (1, 2, 3), (2, 3, 4), (3, 4, 5).

what kind of streak we're looking for matters because it affects both identification of the event of interest and the number of possible events and thus affects our conclusions. Since probability is just counting, we need to make sure of what we're counting.

For this problem, let's say that we're interested in the 3rd one...we want to know whenever 3 days in a row have 1's regardless of what happens before or after. This makes the simulation a bit easier because for a simulation of length M and a sequence of length N, there are M-N+1 such possible sequences.

Let's write a function that identifies these sequences:

In [9]:
def count_sequences( n, data):
    all_sequences = [data[i:i+n] for i in range(len( data)-n+1)]
    streaks = sum([1.0 if sum(xs) == float(n) else 0.0 for xs in all_sequences])
    return streaks, len( all_sequences)

In [10]:
data = simulate_difference_for_n_days( 100000, 0.0001, 0.00011, 30)
print( data)
streak, sequences = count_sequences( 3, data)
print( streak, sequences, streak/sequences)

[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0]
2.0 28 0.07142857142857142


So that's about a 7.1% chance of seeing a streak of 3 days at least once in a 30 day period in a single city. If we want to measure the average, we'd need to re-run the experiment a lot of times:

In [11]:
streaks = []
sequences = []
for i in range( 100):
    data = simulate_difference_for_n_days( 100000, 0.0001, 0.00011, 30)
    streak, sequence = count_sequences( 3, data)
    streaks.append( streak)
    sequences.append( sequence)
print( sum(streaks)/sum(sequences))

0.05035714285714286


So, roughly, there's a 5.0% chance of seeing at least one streak of 3 days in a single city over a 30 day period even if the treatment is better.

But is this what we really want to know? There are 20 cities...we want to know the probability of observing such a streak in at least one of the *20* cities...so the event space is "streak-in-a-city".

In [12]:
def streak_in_a_city( cities, population, control_rate, treatment_rate, streak_length):
    results = []
    for i in range( cities):
        data = simulate_difference_for_n_days(population, control_rate,
                                              treatment_rate, 30)
        streak, sequence = count_sequences( streak_length, data)
        result = 1.0 if streak > 0 else 0.0
        results.append( result)
    return sum(results), cities

In [13]:
streaks, cities = streak_in_a_city( 20, 100000, 0.0001, 0.00011, 3)
print( streaks, cities, streaks/cities)

10.0 20 0.5


In [14]:
streaks = []
experiments = []
for i in range( 100):
    streak, cities = streak_in_a_city( 20, 100000, 0.0001, 0.00011, 3)
    streaks.append( streak)
    experiments.append( cities)
print( sum( streaks)/sum(experiments))

0.6765


As we can see, we expect to see a losing streak of 3 in *some* city even if the treatment is definitely better than control about 67.7% of the time.

Here's an interesting observation. Once we had the probability of 5.0% for a streak of losses in a city, did we need to do a simulation for 20 cities? The answer is, no.

If we let a losing streak be "tails" then the question we're asking about our new model and cities is, if the probability of tails is 5%, and we flip 20 coins simultaneously, what is the probability that we'll see at least one tail? The "at least one" part is what makes it harder. However, if we reframe the question as, what is the probability of seeing 20 heads when tossing 20 coins if the probability of heads is 95%, then we have a draw from a Binomial distribution:

In [15]:
from scipy.stats import binom

In [16]:
binom.pmf(20, 20, 0.95)

0.3584859224085419

If this is the probability of all heads, then we can use the Axioms of Probability to find out the probability of 1 tail or 2 tails or 3 tails, etc, which is "at least one tail". Since the probability of all outcomes is one, and we have the probability of the single outcome we *don't* want, we can simply subtract it from 1 to get the probability of the outcomes we *do* want. And if we take 1 - 0.36, we get 0.64, the probability of "at least one tail". The result is close to what we simulated. You should always be on the lookout for such shortcuts.

Now try solving the problem yourself with different assumptions about what constitutes a streak or even what the true difference between the new model (treatment) and control is (the lift...which can be negative and still be called "lift").